In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import sys

sys.path.append(os.path.abspath("../"))


import faiss
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 400)


load_dotenv()


from src.data_utils import PodcastContainer, load_clean_scores
from src.player_utils import PlayerUtil
from src.utils import get_repo_root

scores = load_clean_scores(["2022-23", "2023-24"])
scores.head(5)

,season_year,game_date,gameId,matchup,teamName,teamSlug,personId,personName,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,fantasyPoints,projectedFantasyPoints,outperformed
360551,2022-23,2022-10-18,22200001,PHI @ BOS,76ers,sixers,200782,pj tucker,33.016667,3,5,0.600,0,2,0.000,0,0,0.0,2,2,4,0,0,1,2,2,6,-6,12.00,17.15,False
360552,2022-23,2022-10-18,22200001,BOS vs. PHI,celtics,celtics,201143,al horford,23.100000,2,7,0.286,2,5,0.400,0,0,0.0,1,4,5,1,0,0,0,4,6,8,13.75,35.15,False
360553,2022-23,2022-10-18,22200001,BOS vs. PHI,celtics,celtics,201933,blake griffin,8.283333,0,2,0.000,0,1,0.000,1,2,0.5,2,3,5,1,0,0,0,3,1,-5,8.75,0.50,True
360554,2022-23,2022-10-18,22200001,PHI @ BOS,76ers,sixers,201935,james harden,37.266667,9,14,0.643,5,9,0.556,12,12,1.0,0,8,8,7,0,0,3,3,35,1,54.00,44.05,True
360555,2022-23,2022-10-18,22200001,PHI @ BOS,76ers,sixers,202699,tobias harris,34.233333,7,14,0.500,3,6,0.500,1,2,0.5,1,1,2,0,3,0,0,3,18,-1,26.50,25.95,True


In [134]:
cont = PodcastContainer(
    {
        "rotowire": get_repo_root() / "data/raw/DG RFB Transcripts/",
    }
)

# begin, end = pd.to_datetime("2023-10-22").date(), pd.to_datetime("2023-10-27").date()
begin, end = pd.to_datetime("2023-10-27").date(), pd.to_datetime("2023-11-05").date()

# Filter podcast episodes by date
podcast_df = cont.get_all_episodes()
podcast_df = podcast_df[
    (podcast_df.publication_date >= begin) & (podcast_df.publication_date < end)
]


df = cont.get_all_episodes()
podcast_df = df[df.file_name.str.contains("week_15")]

display(podcast_df)

,publication_date,file_name,file_path,content,duration,podcast_name
56,2024-01-26,fantasy_basketball_waiver_wire_for_week_15_202324,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome everybody to the award winning rotor w...,2839,rotowire


In [199]:
from src.llm_feature_extractor import FaissFeatureExtractor

title = podcast_df.file_name.iloc[0]
text = podcast_df.content.iloc[0]
podcast_date = podcast_df.publication_date.iloc[0]

fe = FaissFeatureExtractor(title, text, podcast_date)

Loading from disk
Loading from disk


In [200]:
%%time

players = ['giannis antetokounmpo',
 'joel embiid',
 'jayson tatum',
 'damian lillard',
 'tyrese haliburton',
 'lebron james',
 'nikola jokic',
 'kevin durant',
 'luka doncic']


feats = fe.extract_llm_feats()

CPU times: total: 93.8 ms
Wall time: 12.9 s


In [201]:
feats

,personName,increased_playing_time,podcast_date
0,damian lillard,0.0,2024-01-26
1,mitchell robinson,-1.0,2024-01-26
2,isaiah hartenstein,0.0,2024-01-26
3,og anunoby,0.5,2024-01-26
4,kristaps porzingis,-0.5,2024-01-26
5,bam adebayo,0.0,2024-01-26
6,terry rozier,0.5,2024-01-26
7,kyle lowry,-0.5,2024-01-26
8,tyler herro,0.0,2024-01-26
9,jimmy butler,0.0,2024-01-26


In [197]:
dist, indices = fe.index.retrieve_relevant_indices("mitchell robinson", "Increase or decrease in minutes and playing time")
for p in [fe.chunks[i] for i in indices]:
    print(p)
    print()

out the Nuggets last night. 122 to 84 at MSG. UJ Anunoby with 26 points and 6 steals, all Knicks starters, plus or better in point differential. Knicks won this despite still being without Isaiah Hartenstein again. Achilles issued. So he's missed 2 games, and, of course, Robinson's out for the year. The Nuggets did have a day of rest, but it was the end of a 5 game road trip. And Denver did go 3 and 2. Alex and Shannon, any reason for concern for the Nuggets? Yes and no. I mean, they still



In [198]:
dist, indices = fe.index.retrieve_relevant_indices("damian lillard", "Increase or decrease in minutes and playing time")
for p in [fe.chunks[i] for i in indices]:
    print(p)
    print()

ore deserving than Dame Damian Lillard, but solid collection. Probably, you know, arguably, the 10 best players in the league or 10 of the 12 best players in the league are in the starting lineup. So it's hard to really nitpick any of those options. Yeah. I mean, I'm a Bucks fan. I don't I don't hate the idea of either Mitchell or of, Brunson being there. I mean, again, Lillard's been good. Just he's Damian Lillard. He's only gonna be so disappointing to an extent. Ken, you still hate this all

he's Damian Lillard. He's only gonna be so disappointing to an extent. Ken, you still hate this all star game? I remember we used to we you and I did, like, a social media breakout one time where I'm like, I love all star weekend, slam dunk contest. Sign me up. You were like, no. This is a joke. It's a joke. No defense. I don't want anybody getting hurt. It just I don't even like to watch it. It's a good time to do chores around the house. It it definitely has become less and less watchable. I



In [182]:
len(fe.chunks), len(indices)

(100, 72)

In [184]:
len(text), len(" ".join(combined))

(40948, 34353)

In [173]:
" ".join(d[1])

"t. Like, I'm on my phone, you know, and, Accumming, I don't care. Just get something done with this garbage is on the air. Yeah. Yeah. Okay. Well, the I'm I mean, I'm still looking forward to all star weekend, but, mostly for the break. For the lack of injury reporting? Exactly. And, hopefully, we can get some good participants in the, slam dunk contest. I don't even know if that's been out. Have any none of those have been announced, have they? Like, 3 points slam dunk. Yeah. We'll see. Okay. Let's go get some real basketball, Ken. Yeah. Let's go to New York where Spike Lee's Knicks blew out the Nuggets last night. 122 to 84 at MSG. UJ Anunoby with 26 points and 6 steals, all Knicks starters, plus or better in point differential. Knicks won this despite still being without Isaiah Hartenstein again. Achilles issued. So he's missed 2 games, and, of course, Robinson's out for the year. The Nuggets did have a day of rest, but it was the end of a 5 game road trip. And Denver did go 3 and 

In [172]:
len(text)

40948

In [163]:
len(text)

40948

In [138]:
p = [p for p, _ in d]


['giannis antetokounmpo',
 'joel embiid',
 'jayson tatum',
 'damian lillard',
 'tyrese haliburton',
 'lebron james',
 'nikola jokic',
 'kevin durant',
 'luka doncic',
 'shai gilgeous-alexander',
 'donovan mitchell',
 'jalen brunson',
 'og anunoby',
 'isaiah hartenstein',
 'mitchell robinson',
 'terry rozier',
 'kristaps porzingis',
 'bam adebayo',
 'kyle lowry',
 'tyler herro',
 'jimmy butler',
 'josh richardson',
 'caleb martin',
 'duncan robinson',
 'hawkeye',
 'highsmith',
 'hawkeyes',
 'harrison barnes',
 'andrew wiggins',
 'kevin huerter',
 'keegan murray',
 'jabari walker',
 'james clutch',
 'grayson allen',
 'gary terry',
 'bilal koulibaly',
 'tyus jones',
 'jordan poole',
 'deni avdija',
 'kyle kuzma',
 'nick richards',
 'mark williams',
 'jonathan kuminga',
 'draymond green',
 'stephen curry',
 'kirk',
 'moses moody',
 'jon conchar',
 'luke kennard',
 'kembe mutombo',
 'vince williams',
 'brandon miller',
 'lamelo ball',
 'miles bridges',
 'gordon hayward',
 'sam merrill',
 'd

In [149]:
set(p) - set(scores.personName.unique())

{'bilal koulibaly',
 'cam johnson',
 'chris middleton',
 'dennis schroeder',
 'emmanuel quickley',
 'gary terry',
 'hawkeye',
 'hawkeyes',
 'highsmith',
 'jacob pirtle',
 'james clutch',
 'jared allen',
 'jon conchar',
 'jontay murray',
 'kembe mutombo',
 'kirk',
 'luca fontecchio',
 'luke cornett',
 'terrence mann',
 'tim hardaway',
 'vince williams'}

In [146]:
len(p)

96

In [145]:
scores[scores.personName.str.contains('tim harda')]

,season_year,game_date,gameId,matchup,teamName,teamSlug,personId,personName,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,fantasyPoints,projectedFantasyPoints,outperformed
360897,2022-23,2022-10-19,22200013,DAL @ PHX,mavericks,mavericks,203501,tim hardaway jr,25.016667,3,10,0.300,1,5,0.200,2,2,1.000,0,2,2,1,1,0,2,1,9,-2,14.00,16.70,False
361482,2022-23,2022-10-22,22200034,DAL vs. MEM,mavericks,mavericks,203501,tim hardaway jr,25.883333,3,10,0.300,2,6,0.333,8,8,1.000,0,2,2,2,0,0,0,1,16,23,21.50,15.05,True
362343,2022-23,2022-10-27,22200066,DAL @ BKN,mavericks,mavericks,203501,tim hardaway jr,30.283333,6,13,0.462,4,8,0.500,2,2,1.000,0,2,2,1,0,2,1,1,18,-1,25.50,15.65,True
362870,2022-23,2022-10-29,22200086,DAL vs. OKC,mavericks,mavericks,203501,tim hardaway jr,22.183333,2,7,0.286,1,3,0.333,3,4,0.750,0,2,2,1,1,0,2,3,8,3,13.00,17.00,False
363054,2022-23,2022-10-30,22200093,DAL vs. ORL,mavericks,mavericks,203501,tim hardaway jr,27.116667,5,15,0.333,3,9,0.333,8,8,1.000,0,1,1,3,1,0,0,2,21,9,28.75,17.10,True
363632,2022-23,2022-11-02,22200115,DAL vs. UTA,mavericks,mavericks,203501,tim hardaway jr,20.766667,1,9,0.111,1,6,0.167,0,0,0.000,0,1,1,2,0,0,1,2,3,-2,6.75,20.55,False
363933,2022-23,2022-11-04,22200127,DAL vs. TOR,mavericks,mavericks,203501,tim hardaway jr,19.100000,2,8,0.250,2,6,0.333,0,0,0.000,0,2,2,3,1,0,2,2,6,-6,14.00,19.10,False
364638,2022-23,2022-11-07,22200155,DAL vs. BKN,mavericks,mavericks,203501,tim hardaway jr,13.800000,4,8,0.500,2,4,0.500,1,2,0.500,0,1,1,1,0,0,0,0,11,6,13.75,17.60,False
364792,2022-23,2022-11-09,22200161,DAL @ ORL,mavericks,mavericks,203501,tim hardaway jr,22.183333,2,10,0.200,1,7,0.143,0,0,0.000,0,2,2,2,2,0,0,0,5,-3,14.50,15.25,False
365076,2022-23,2022-11-10,22200172,DAL @ WAS,mavericks,mavericks,203501,tim hardaway jr,28.083333,5,12,0.417,5,11,0.455,1,2,0.500,1,1,2,1,0,0,0,0,16,-7,20.00,15.55,True


In [110]:
player_mapping = fe.index.player_chunk_mapping
text_chunks = fe.index.text_chunks

In [104]:
text_chunks[31]

"surprised if the if the injury drags on maybe a little bit longer than than was hoped just it's a pretty serious injury. So the current current stance is he was cleared for basketball activity around January 19th, ruled out of the 4 game road trip that Cleveland was on at the time. Yeah. They're gonna return to home January 29th against the Clippers. So we're looking at Monday night. He could return as soon as next week. But did they also I they don't even need to rush him back. They don't lose anymore. So, like, he's been playing great. Jared Allen has been awesome lately, down low without Mobley. Anyway, we digress. Whose turn is it for a pick here? It's my we digress. Whose turn is it for a pick here? I think it's my turn. I think it is. You got a second long shot, don't you, Alex? I do. Sorry. I was looking up Brandon Miller stats. I was like, how is he? Brandon Miller hasn't missed a shot at the rim in the past 4 games, by the way. 12 of 12, and getting to the foul line. I owe"

In [84]:
relevant_chunks = [text_chunks[idx] for idx in player_mapping['dennis schroder']]
relevant_chunks

["RJ Barrett, Dennis Schroeder as value plays the same at the same time. RJ Barrett is probably going to be checked by Paul George or Kawhi Leonard for a decent chunk of the game or Terrence man. So I would be a little bit worried about that. I would almost just prefer Dennis Schroder at 59100. But to me at this again, I don't feel like 5,900 is a is like a steal for Dennis Schroder. You know, I don't feel like I'm slotting him my my lineup and getting almost guaranteed value. Now even before they acquired Quickley, Schroder was oftentimes in that $6,000 range. So I agree. He is a a better value, on DraftKings than he is FanDuel for tonight's slate. 5,200 for Schroeder on DraftKings. I like him considerably more at 5,200 than 59 on FanDuel. But I think if he's if he's announced as a starter, he is in play. Another guard from a different game that I like quite a bit right in that same range. 54100 from Markelle Fultz on FanDuel. Yeah. He is back in the starting lineup, played 29 minutes

In [111]:
rc = fe.index.retrieve_relevant_chunks('andrew wiggins', 'minutes playing time')
rc


TypeError: combine_chunks_with_overlap() missing 1 required positional argument: 'overlap_length'

In [108]:
rc

["gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,",
 

In [105]:
relevant_chunks = [text_chunks[idx] for idx in player_mapping['andrew wiggins']]
relevant_chunks
# d = combine_chunks_with_overlap(relevant_chunks, overlap_length = 200)

["gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,",
 

In [101]:
for s in d:
    print(s)
    print()

gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,

and 

In [82]:
for c in relevant_chunks:
    print(c)
    print()

gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,

and 

In [62]:
r = ner.extract_all_players(chunks[9])
r

['Hawkeyes', 'Harrison Barnes', 'Andrew Wiggins']

In [58]:
r.players

['Harrison Barnes', 'Andrew Wiggins']

In [63]:
len(chunks)

51

In [52]:
chunks[9]

"gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,"

In [46]:
chunks[9]

"gonna take some time for the chemistry to build. I'll say this for the heat. You know, they're on 5 game losing streak, and Hawkeyes has been out for 6 games. So he's missed all 5 of those losses. I think he's a critical part of this heat rotation. They should move Hawkeyes in the starting lineup in my opinion, or at least make sure he's getting plus 27 minutes a game. I think there's a good chance he starts over high. Smith wants healthy. Yep. Alright. Let's go to Golden State. Last night, the Kings beat the warriors in a thriller. Kings, Kings 1, 134, 133. Harrison Barnes, 39 points. His second straight, 30 plus point scoring night. And fantasy wise, Wiggins got to start again for the warriors. He went 174 and 4 and 31 minutes. So Wiggins slowly earning legit minutes for the warriors. Shannon's, I think you pointed out last week's show, but back to the Kings after a 4 game slide, they won 2 in a row. Is the arrow pointing up for Sacramento, gentlemen? You know, sort of, I mean,"

In [34]:
rel_chunks = fe.index.retrieve_relevant_chunks(
    "Duncan Robinson from the Knicks minutes playtime"
)
rel_chunks

[(0.3326186,
  "Have any none of those have been announced, have they? Like, 3 points slam dunk. Yeah. We'll see. Okay. Let's go get some real basketball, Ken. Yeah. Let's go to New York where Spike Lee's Knicks blew out the Nuggets last night. 122 to 84 at MSG. UJ Anunoby with 26 points and 6 steals, all Knicks starters, plus or better in point differential. Knicks won this despite still being without Isaiah Hartenstein again. Achilles issued. So he's missed 2 games, and, of course, Robinson's out for the year. The Nuggets did have a day of rest, but it was the end of a 5 game road trip. And Denver did go 3 and 2. Alex and Shannon, any reason for concern for the Nuggets? Yes and no. I mean, they still can't win road games. They're 14 and 11 on the road. We always know their better at home. There's a championship hangover thing that makes me not as worried where I think, you know, they they know they need to coast this year compared to last year a little bit more. At the same time, to 

In [23]:
dist

NameError: name 'dist' is not defined

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

title = podcast_df.file_name.iloc[0]
text = podcast_df.content.iloc[0]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(text)

In [14]:
chunks[0]

"Welcome everybody to the award winning rotor wire fantasy basketball podcast brought to you by Underdog Fantasy and FanDuel. It is Friday, January 26, Alex Barutha here with Shannon McEwen and Ken K train crates. All aboard. We had the All Star starters announced last night in the East, yanis, embiid Tatum, Lillard and Halliburton in the West LeBron. That's James. Jokic KD, Luca and SGA Shannon as is tradition. Is there anybody that you think was either snubbed or maybe you just would have preferred to see as an all star or one person that you think maybe doesn't deserve to be there. Yeah. I I think in general, all 10 all 10 starters, definitely deserve, the the the nomination. I the only one I really have any qualms about is Lillard. He's having a fine year for Milwaukee in his 1st year as a buck, but there are 2 east cars. Yeah. I I actually, Donovan Mitchell would be my number one choice to to replace Lillard and then Jalen Brunson. So so one of those two guards, I think, would be"

In [15]:
chunks[1]

"as a buck, but there are 2 east cars. Yeah. I I actually, Donovan Mitchell would be my number one choice to to replace Lillard and then Jalen Brunson. So so one of those two guards, I think, would be more deserving than Dame Damian Lillard, but solid collection. Probably, you know, arguably, the 10 best players in the league or 10 of the 12 best players in the league are in the starting lineup. So it's hard to really nitpick any of those options. Yeah. I mean, I'm a Bucks fan. I don't I don't hate the idea of either Mitchell or of, Brunson being there. I mean, again, Lillard's been good. Just he's Damian Lillard. He's only gonna be so disappointing to an extent. Ken, you still hate this all star game? I remember we used to we you and I did, like, a social media breakout one time where I'm like, I love all star weekend, slam dunk contest. Sign me up. You were like, no. This is a joke. It's a joke. No defense. I don't want anybody getting hurt. It just I don't even like to watch it."